# **Importy**

In [1]:
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import drive
from sklearn.feature_selection import mutual_info_regression

# **Načítanie a dát**
Údaje by mali byť vo formáte CSV alebo XLSX s génmi (Geneid) ako riadkami a replikátmi BAM ako stĺpcami.

In [ ]:
# Pripojenie na Disk Google a načítanie dátového súboru

# Mount Google Drive
drive.mount("/content/drive", force_remount=True)

# Definovanie cesty k dátovému súboru
root_dir = "/content/drive/MyDrive/"
base_dir = os.path.join(root_dir, "BP/DATA/")
data_file = 'MBF1.xlsx'  # Napríklad data.xlsx

# Kontrola, či je súbor CSV alebo XLSX a načitanie do pandas DataFrame
file_path = os.path.join(base_dir, data_file)
if data_file.endswith('.csv'):
    data_df = pd.read_csv(file_path, delimiter=",", index_col=0, header=0)
elif data_file.endswith('.xlsx'):
    data_df = pd.read_excel(file_path, index_col=0, header=0)
else:
    raise ValueError("Nepodporovaný formát súboru. Poskytnite súbor CSV alebo XLSX.")

# Zobrazenie DataFrame
data_df

# **Detekcia potenciálne koregulovaných génov v celom datasete**

## Transponovanie dát pre výpočet korelačného koeficientu a vzájomnej informácie

In [3]:
gene_expression_data_T = data_df.T

## **Pearsonov korelačný koeficient**

In [ ]:
# Výpočet korelačnej matice pomocou Pearsonovej metódy
corr_matrix = gene_expression_data_T.corr()

# Nahradenie NaN hodnôt nulami
corr_matrix = corr_matrix.fillna(0)

# Nastavenie diagonály na 1 pomocou numpy.fill_diagonal
np.fill_diagonal(corr_matrix.values, 1)

# Uloženie korelačnej matice do súboru .csv
corr_matrix.to_csv(os.path.join(base_dir, 'corr_matrix.csv'))

# Vizualizácia korelačnej matice - heatmapa
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, cmap='viridis', annot=False)
plt.title('Heatmapa korelačnej matice')
plt.xlabel('Gény')
plt.ylabel('Gény')
plt.show()

Nahradenie NaN hodnôt:

Ak sú hodnoty expresie pre niektorý pár génov konštantné (napríklad, ak sú všetky hodnoty pre gény rovnaké) pri výpočte Pearsonovho korelačného koeficienta dôjde k deleniu nulou, čo vedie k hodnote NaN. To je preto, že štandardná odchýlka (v menovateli vzorca) je nulová pre konštantné hodnoty. Preto som sa po preštudovaní dokumentácie (https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.corr.html) a zistení, že funkcia .corr() dáva NaN tam, kde nie je žiadna korelácia, rozhodla nahradiť túto nulovú závisloť matematicky a teda 0.
Následne definujem diagonálu 1, pretože korelácia toho istého génu so sebou samým bude vždy maximálna.

## **Vzájomná informácia (mutual information)**



In [ ]:
# Vytvorenie prázdneho DataFrame pre vzájomnú informáciu
mi_matrix = pd.DataFrame(index=gene_expression_data_T.columns, columns=gene_expression_data_T.columns, data=0.0)

# Výpočet vzájomnej informácie medzi stĺpcami (génmi)
for col1 in gene_expression_data_T.columns:
    for col2 in gene_expression_data_T.columns:
        mi_matrix.loc[col1, col2] = mutual_info_regression(gene_expression_data_T[[col1]], gene_expression_data_T[col2])[0]

# Nastavenie diagonály na maximum pomocou numpy.fill_diagonal
max_mi_value = mi_matrix.max().max()
np.fill_diagonal(mi_matrix.values, max_mi_value)

# Uloženie matice vzájomnej informácie do súboru .csv
mi_matrix.to_csv(os.path.join(base_dir, 'mi_matrix.csv'))

# Vizualizácia matice vzájomnej informácie - heatmapa
plt.figure(figsize=(10, 8))
sns.heatmap(mi_matrix, cmap='viridis', annot=False)
plt.title('Heatmapa matice vzájomnej informácie')
plt.xlabel('Gény')
plt.ylabel('Gény')
plt.show()

Nastavenie digonály na maximum:

Ide o podobný prípad ako s korelačnou analýzou a použitím funkcie .corr(). Aj funkcia pre výpočet vzájomnej informácie .mutual_info_regression() (https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.mutual_info_regression.html) pri určovaní závislosti génov s konštantnými hodnotami dáva veľmi nízke hodnoty a opäť je z elementárneho hľadiska samozrejmosťou výsledná vzájomná informácia maximálna pre gén so sebou samým.

## **Prahovanie korelačnej matice a matice vzájomnej informácie**

In [ ]:
# Vytvorenie prázdneho DataFrame pre prahovanú maticu korelačného koeficientu
corr_threshold_matrix = pd.DataFrame(np.zeros_like(corr_matrix), index=corr_matrix.index, columns=corr_matrix.columns)

# Definovanie prahovej hodnoty
threshold = 0.5

# Vytvorenie podmienky na zistenie, ktoré hodnoty v corr_matrix sú väčšie alebo rovné prahovej hodnote
condition = (corr_matrix >= threshold)

# Nastavenie hodnôt v corr_threshold_matrix na 1 tam, kde podmienka platí
corr_threshold_matrix[condition] = 1

# Uloženie prahovanej korelačnej matice do súboru .csv
corr_threshold_matrix.to_csv(os.path.join(base_dir, 'corr_threshold_matrix.csv'))

# Vizualizácia prahovanej korelačnej matice
plt.figure(figsize=(8, 8))
sns.heatmap(corr_threshold_matrix, cmap='binary', annot=True, cbar=False)
plt.title('Binárny obraz prahovanej korelačnej matice')
plt.xlabel('Gény')
plt.ylabel('Gény')
plt.show()

In [ ]:
# Vytvorenie prázdneho DataFrame pre prahovanú maticu vzájomnej informácie
mi_threshold_matrix = pd.DataFrame(np.zeros_like(mi_matrix), index=mi_matrix.index, columns=mi_matrix.columns)

# Definovanie prahovej hodnoty
threshold = np.mean(mi_matrix.values)

# Vytvorenie podmienky na zistenie, ktoré hodnoty v mi_matrix sú väčšie alebo rovné prahovej hodnote
condition = (mi_matrix >= threshold)

# Nastavenie hodnôt v mi_threshold_matrix na 1 tam, kde podmienka platí
mi_threshold_matrix[condition] = 1

# Uloženie prahovanej matice vzájomnej informácie do súboru .csv
mi_threshold_matrix.to_csv(os.path.join(base_dir, 'mi_threshold_matrix.csv'))

# Vizualizácia prahovanej matice vzájomnej informácie
plt.figure(figsize=(8, 8))
sns.heatmap(mi_threshold_matrix, cmap='binary', annot=True, cbar=False)
plt.title('Binárny obraz prahovanej matice vzájomnej informácie')
plt.xlabel('Gény')
plt.ylabel('Gény')
plt.show()

## **Výsledná prieniková matica**

In [ ]:
# Vytvorenie prázdneho DataFrame pre prienikovú maticu
result_matrix = pd.DataFrame(np.zeros_like(corr_threshold_matrix), index=corr_threshold_matrix.index, columns=corr_threshold_matrix.columns)

# Výpočet prieniku dvoch prahovaných matíc
result_matrix = (mi_threshold_matrix == 1) & (corr_threshold_matrix == 1)

# Uloženie výslednej prienikovej matice do súboru .csv
result_matrix.to_csv(os.path.join(base_dir, 'result_matrix.csv'))

# Vizualizácia výslednej prienikovej matice
plt.figure(figsize=(8, 8))
sns.heatmap(result_matrix, cmap='binary', annot=True, cbar=False)
plt.title('Binárny obraz prienikovej matice')
plt.xlabel('Gény')
plt.ylabel('Gény')
plt.show()

# **Detekcia potenciálne koregulovaných génov pre konkrétny gén (Geneid)**

In [ ]:
# Definovanie cieľového génu pre analýzu
target_gene = "AT1G01460"  # Napríklad AT1G01460

# Kontrola, či cieľový gén existuje v dátach
if target_gene not in data_df.index:
    raise ValueError(f"Gén {target_gene} sa v údajoch nenašiel.")

# Transponovanie dát pre výpočet korelačného koeficientu a vzájomnej informácie
gene_expression_data_T = data_df.T

# Výpočet korelačnej matice pre cieľový gén voči všetkým ostatným génom pomocou Pearsonovej metódy
corr_matrix = pd.DataFrame(index=gene_expression_data_T.columns, columns=[target_gene])

for gene in gene_expression_data_T.columns:
    corr_matrix.loc[gene, target_gene] = gene_expression_data_T[target_gene].corr(gene_expression_data_T[gene])

# Uloženie korelačnej matice do súboru .csv
corr_matrix.to_csv(os.path.join(base_dir, 'gene_corr_matrix.csv'))

# Výpočet matice vzájomnej informácie pre cieľový gén voči všetkým ostatným génom
mi_matrix = pd.DataFrame(index=gene_expression_data_T.columns, columns=[target_gene])

for gene in gene_expression_data_T.columns:
    mi_value = mutual_info_regression(gene_expression_data_T[[target_gene]], gene_expression_data_T[gene])[0]
    mi_matrix.loc[gene, target_gene] = mi_value

# Uloženie matice vzájomnej informácie do súboru .csv
mi_matrix.to_csv(os.path.join(base_dir, 'gene_mi_matrix.csv'))

# Vytvorenie prázdneho DataFrame pre prahovanú maticu korelačného koeficientu
corr_threshold_matrix = pd.DataFrame(np.zeros_like(corr_matrix), index=corr_matrix.index, columns=corr_matrix.columns)

# Definovanie prahovej hodnoty
threshold = 0.8

# Vytvorenie podmienky na zistenie, ktoré hodnoty v corr_matrix sú väčšie alebo rovné prahovej hodnote
condition = (corr_matrix >= threshold)

# Nastavenie hodnôt v corr_threshold_matrix na 1 tam, kde podmienka platí
corr_threshold_matrix[condition] = 1

# Uloženie prahovanej korelačnej matice do súboru .csv
corr_threshold_matrix.to_csv(os.path.join(base_dir, 'gene_corr_threshold_matrix.csv'))

# Vytvorenie prázdneho DataFrame pre prahovanú maticu vzájomnej informácie
mi_threshold_matrix = pd.DataFrame(np.zeros_like(mi_matrix), index=mi_matrix.index, columns=mi_matrix.columns)

# Definovanie prahovej hodnoty
threshold = np.mean(mi_matrix.values)

# Vytvorenie podmienky na zistenie, ktoré hodnoty v mi_matrix sú väčšie alebo rovné prahovej hodnote
condition = (mi_matrix >= threshold)

# Nastavenie hodnôt v mi_threshold_matrix na 1 tam, kde podmienka platí
mi_threshold_matrix[condition] = 1

# Uloženie prahovanej matice vzájomnej informácie do súboru .csv
mi_threshold_matrix.to_csv(os.path.join(base_dir, 'gene_mi_threshold_matrix.csv'))

# Vytvorenie prázdneho DataFrame pre prienikovú maticu
result_matrix = pd.DataFrame(np.zeros_like(corr_threshold_matrix), index=corr_threshold_matrix.index, columns=corr_threshold_matrix.columns)

# Výpočet prieniku dvoch prahovaných matíc
result_matrix = (mi_threshold_matrix == 1) & (corr_threshold_matrix == 1)

# Odstránenie cieľového génu z výslednej matice
result_matrix_no_target = result_matrix.drop(target_gene)

# Uloženie výslednej prienikovej matice do súboru .csv
result_matrix_no_target.to_csv(os.path.join(base_dir, 'gene_result_matrix.csv'))

# Výpis výslednej matice ako DataFrame
print("Výsledná matica koregulovaných génov so zadaným génom:\n")
print(result_matrix_no_target)

# Spočítanie koregulovaných génov vo výslednej matici
total_number_of_ones = int(result_matrix_no_target.values.sum())

print(f"Počet koordinovane regulovaných génov s {target_gene}: {total_number_of_ones}")